In [1]:
import pandas as pd

# Load the two Excel files to inspect their contents
file1_path = "./CLG Machine shop production master file 03.xlsx"
file2_path = "./CLG.xlsx"

# Load all sheets to understand their structure
file1_sheets = pd.ExcelFile(file1_path).sheet_names
file2_sheets = pd.ExcelFile(file2_path).sheet_names

file1_sheets, file2_sheets

(['Sheet1'], ['Sheet1'])

In [2]:
# Load the data from Sheet1 of both files
file1_data = pd.read_excel(file1_path, sheet_name="Sheet1")
file2_data = pd.read_excel(file2_path, sheet_name="Sheet1")

# Display the first few rows of each file to understand their structure
file1_data.head(), file2_data.head()

(     SKU #  PO Quantity  Sequence  Component  Total Comp. Qty.  \
 0  7677114           58          1   TOP PLY                58   
 1  7677114           58          1   TOP PLY                58   
 2  7677114           58          1   TOP PLY                58   
 3  7677114           58          1   TOP PLY                58   
 4  7677114           58          1   TOP PLY                58   
 
    Min Batch Size to move to next Machine  Changeover time in Min  \
 0                                      20                       0   
 1                                      20                       0   
 2                                      20                       0   
 3                                      20                       0   
 4                                      20                       0   
 
               Machine Route  Process time in Minutes/Part  
 0                 BEAM SAW                      12.778345  
 1                HOT PRESS                      20.

# New section

File 1 (CLG Machine shop production master file 03.xlsx):
SKU #: Identifier for the product.
PO Quantity: Order quantity for each product.
Sequence: Sequence number in the manufacturing process.
Component: Component name for the process.
Total Comp. Qty.: Total quantity of the component to be produced.
Min Batch Size to Move to Next Machine: Minimum production batch size to proceed.
Changeover Time in Min: Time required for setup between operations.
Machine Route: Machine names for processing.
Process Time in Minutes/Part: Time required to process one part.
File 2 (CLG.xlsx):
Similar to File 1, but without the "SKU #" and "Min Batch Size to Move to Next Machine" columns.



In [3]:
import pandas as pd
import numpy as np

# Load data
file_path = "./CLG Machine shop production master file 03.xlsx"
data = pd.read_excel(file_path, sheet_name="Sheet1")
data.head()

,SKU #,PO Quantity,Sequence,Component,Total Comp. Qty.,Min Batch Size to move to next Machine,Changeover time in Min,Machine Route,Process time in Minutes/Part
0,7677114,58,1,TOP PLY,58,20,0,BEAM SAW,12.778345
1,7677114,58,1,TOP PLY,58,20,0,HOT PRESS,20.541667
2,7677114,58,1,TOP PLY,58,20,0,FILING,18.125000
3,7677114,58,1,TOP PLY,58,20,0,BEAM SAW,12.778345
4,7677114,58,1,TOP PLY,58,20,0,WIDE BELT SANDER (COSTA),12.962121


In [4]:

# Data Cleaning and Preparation
data = data.dropna()  # Drop any rows with missing values

# Ensure column names are consistent
data.columns = [
    "SKU", "PO_Quantity", "Sequence", "Component", "Total_Comp_Qty",
    "Min_Batch_Size", "Changeover_Time", "Machine", "Process_Time_Per_Part"
]


In [5]:

# Convert necessary columns to numeric types for calculations
data["PO_Quantity"] = data["PO_Quantity"].astype(int)
data["Sequence"] = data["Sequence"].astype(int)
data["Total_Comp_Qty"] = data["Total_Comp_Qty"].astype(int)
data["Min_Batch_Size"] = data["Min_Batch_Size"].astype(int)
data["Changeover_Time"] = data["Changeover_Time"].astype(float)
data["Process_Time_Per_Part"] = data["Process_Time_Per_Part"].astype(float)

# Group by SKU and Sequence to analyze the process flow
process_flow = data.groupby(["SKU", "Sequence"]).agg({
    "Component": "first",
    "Machine": "first",
    "Process_Time_Per_Part": "sum",
    "Changeover_Time": "sum",
    "Total_Comp_Qty": "max",
    "Min_Batch_Size": "max"
}).reset_index()


In [6]:

# Function to simulate production schedule
def simulate_production(process_flow, shifts=3, hours_per_shift=8):
    """
    Simulates a shift-wise production schedule.

    Args:
        process_flow (DataFrame): Process flow data.
        shifts (int): Number of shifts per day.
        hours_per_shift (int): Hours in each shift.

    Returns:
        dict: Simulation results including bottlenecks, lead times, and machine load.
    """
    total_hours_per_day = shifts * hours_per_shift
    results = []

    # Iterate over the process flow
    for _, row in process_flow.iterrows():
        component = row["Component"]
        machine = row["Machine"]
        process_time = row["Process_Time_Per_Part"]
        changeover_time = row["Changeover_Time"]
        total_qty = row["Total_Comp_Qty"]
        min_batch_size = row["Min_Batch_Size"]

        # Calculate time required for the process
        batches = np.ceil(total_qty / min_batch_size)
        total_process_time = batches * process_time + changeover_time

        # Calculate shifts and days required
        total_time = total_process_time / 60  # Convert minutes to hours
        days_required = np.ceil(total_time / total_hours_per_day)

        results.append({
            "Component": component,
            "Machine": machine,
            "Total_Time_Hours": total_time,
            "Days_Required": days_required
        })

    return pd.DataFrame(results)


In [7]:

# Simulate production
simulation_results = simulate_production(process_flow)

# Identify bottlenecks
bottleneck_machines = simulation_results.sort_values(by="Total_Time_Hours", ascending=False).head(1)

# Longest lead time
longest_lead_time = simulation_results["Days_Required"].max()

# Machine loading plan
machine_loading = simulation_results.groupby("Machine").agg({
    "Total_Time_Hours": "sum"
}).reset_index()

# Outputs
print("Simulation Results:")
print(simulation_results)
print("\nBottleneck Machines:")
print(bottleneck_machines)
print("\nLongest Lead Time (Days):", longest_lead_time)
print("\nMachine Loading Plan:")
print(machine_loading)


Simulation Results:
              Component              Machine  Total_Time_Hours  Days_Required
0               TOP PLY            BEAM SAW          11.298156            1.0
1              BASE PLY            BEAM SAW           6.939819            1.0
2        INNER SIDE PLY            BEAM SAW           8.040541            1.0
3        OUTER SIDE PLY            BEAM SAW          11.033228            1.0
4             PARTITION            BEAM SAW           3.000505            1.0
5        DOOR PINE WOOD  DOUBLE SIDE PLANNER          6.917438            1.0
6          TAPPER FRAME  DOUBLE SIDE PLANNER          7.590972            1.0
7        TOP FRAME (SG)  DOUBLE SIDE PLANNER          8.090972            1.0
8       SIDE FRAME (DG)  DOUBLE SIDE PLANNER         10.435234            1.0
9            BACK FRAME  DOUBLE SIDE PLANNER          6.624421            1.0
10  BUTTAN 1 MANGO WOOD  DOUBLE SIDE PLANNER          3.171603            1.0
11          FRONT PATTI  DOUBLE SIDE PLANNER